# 4. Dynamics

In this Notebook we will introduce the __dynamic model__, also commonly referred as the __dynamics__. The dynamic model relates the forces/torques applied in the robot's actuators to the joint positions, velocities and accelerations. Similarily to the geometric and kinematic model, there are two kinds of dynamic models:

- __Direct Dynamic Model (DDM)__ : given joint positions $q$, velocities $\dot{q}=\frac{dq}{dt}$ and torques $\tau$, it enables to calculate the resulting joint accelerations $\ddot{q}=\frac{d \dot{q}}{dt}$
- __Inverse Dynamic Model (IDM)__ : given a joint positions $q$, velocities $\dot{q}=\frac{dq}{dt}$ and accelerations $\ddot{q}=\frac{d \dot{q}}{dt}$, it enables to calculate the torques $\tau$

__Note__ : you may want to review [Newton's Laws of Motion](https://en.wikipedia.org/wiki/Newton%27s_laws_of_motion) and the notion of [torque](https://www.grc.nasa.gov/WWW/K-12/airplane/torque.html) before starting this Notebook

In [1]:
## This is the implementation of a 1 degree of freedom arm
## Author: Avadesh Meduri
## Date : 21/06/2020

import numpy as np
from matplotlib import pyplot as plt

# these packages for animating the robot env
import IPython
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation


In order to execute some desired motion, we need to tell each actuator which contribution it should bring to the overall joint space motion. The commands ultimately sent to the robot's actuators are the typically motor currents (eletrical actuation) or pistons fluid pressure (hydraulic actuation). But on a higher level, what causes these motion are forces and torques output by the actuators, regardless of how they internally function. So for now, we will disgard the low-level actuation process and assume "ideal" actuators than generate directly a force or a torque when commanded so.   

## 1R Robot : Dynamic Modeling

Now let us take again the example of the 1R robot. 

__Important note: the angle convention used in the following (i.e. $q$ measured w.r.t. $y$-axis) is different than the one we used in the previous notebooks on modeling (i.e. $q$ measured w.r.t. $x$-axis). The present convention is the most common for analyzing the simple pendulum system__

<!-- For Colab NB -->
[Pendulum image (click here)](https://drive.google.com/uc?id=1fNrxgrN7MREa6VjCLJrlIGtEfN_febv3)

In [4]:
# # If full github repo has been downloaded locally
# from IPython.display import Image
# # Display the 1R robot
# Image("pendulum.png")

Neglecting the details of the actuation, the rotational joint at $O$ can be viewed as a pure torque source. The torque $\tau$, together with the effect of gravity acting at point $G$ (tip of the rod), generates an angular acceleration $\ddot{q}$. These two quantities satisfy [Euler's law](https://en.wikipedia.org/wiki/Euler%27s_laws_of_motion) :

$$
\tau + \sum \vec{M}_O(\vec{F}) = I \ddot{q}
$$

where 
- $ \sum \vec{M}_O(\vec{F})$ represent the sum of the moments of external forces applied on the robot, taken with respect to the base / origin point $O$
- $I$ is the [inertia](https://en.wikipedia.org/wiki/List_of_moments_of_inertia) of the link (in $kg.m^2.s^2$)

Here the only external force applied on the robot is gravity, acting at the center of mass $G$ and given by $\vec{F}_{gravity} = -mg \vec{y}$ (red arrow in the picture). 

__Question__ : Assuming that $\|\vec{OG}\| = L$, what it the moment due to gravity about $O$ ? 

Hint: Use the definition of the [moment of a force about a point](https://www.miniphysics.com/moment-of-force.html)

__Question__ : Knowing the moment of gravity, how to deduce the DDM / IDM ?


__Question__ : The class below implements the DDM / IDM of the 1R robot. Complete the code. Use $I=\frac{1}{3}mL^2$.

In [5]:
class DynamicModel1R:
    
    def __init__(self, length, mass):
        '''
        This function initialises the class OneDOFManipulatorEnv
        Input:
            length : lenght of the arm
            mass : mass of the rod
        '''
        
        self.length = length
        self.dt = 0.001 # discretization step in seconds
        self.g = 9.81 # gravity vector
        self.m = mass
        # Computing the intertia of the rod about an axis
        # fixed at the end (1/3)ml^2
        # >> Implement here the class attribute for inertia I
        
    def DDM(self, q, v, tau):
        '''
        This function computes the direct dynamics model (DDM) of the manipulator given
        the current state of (Joint Position q, Joint Velocity v) and torque tau
        Input:
            q   : joint position 
            v   : joint velocity
            tau : joint torque
        Output:
            a   : joint acceleration
        '''
        
        # converting q from degree to radians
        # return acceleration
        pass

    def IDM(self, q, v, a):
        '''
        This function computes the inverse dynamics model (IDM) of the manipulator given
        the current state of (Joint Position q, Joint Velocity v) and acceleration a
        Input:
            q : joint position 
            v : joint velocity
            a : joint acceleration
        Output:
            tau : joint torque
        '''
        
        # converting from degree to radians
        # return torque
        pass

## Continuous-Time Dynamics 

The DDM equation belongs to a well-known family of equations in mathematics: it is a _continuous-time_ Ordinary [Differential Equation](https://www.mathsisfun.com/calculus/differential-equations.html) (ODE). It means that it describes the evolution of the robot (or the _system_) throughout time, represented by a continuous variable $t$ (i.e. a real number: $t\in\mathbb{R}$). The dynamics of the 1R robot implemented above can indeed be written under the general form 

$$
\dot x(t) = f(x(t),u(t))~~~\text{for}~t\in\mathbb{R} 
$$

where $x=[q,v]^T$ is called the _state_ (joint position, velocity), $u=\tau$ is called the _control input_ (torque) and $t$ is the _continuous time variable_. Given a control input __trajectory__ $u(t)$ (i.e. a joint torque signal), a _solution_ to this ODE is a state __trajectory__ $x(t)$ (i.e. joint position and velocity signals) that satisfies the above equation, i.e. $\frac{dx}{dt}=f(x(t),u(t))$.

__Question__ : Write the continuous-time ODE in matrix form using the given definitions of the state and control variables. It should look like 

$$
\dot x = \begin{bmatrix} 
            f_1(x) \\
            f_2(x,u)
         \end{bmatrix}
$$ 

where $f_1,f_2$ are functions of $q,v,\tau$ to be determined from the DDM.

__Hint__ : The left-hand side is $\dot x = [\dot q, \dot v]^T$, and we know that by definition $\dot v = \ddot q$ (joint acceleration). Use the DDM equation found earlier (and some logic) to deduce what $f_1,f_2$ should be in the right-hand side.

## Discretization 

In order to be able to simulate the system in a program, we need to transform this continuous-time ODE into a _discrete-time_ ODE where $t=1,2,...$ (i.e. where $t$ is an integer). This procedure is called _discretization_ (or _integration_) and is crucial to understand how simulations work. The [Euler discretization](https://en.wikipedia.org/wiki/Euler_method) with timestep $\Delta t>0$ (expressed in seconds) transforms the ODE into  

$$
x_{n+1} = x_n + f(x_n,u_n)\Delta t~~~\text{for}~n=0,1,2,...
$$

which is a good approximation of the continuous time model whenever $\Delta t$ is small. Given a control input __sequence__ $u_0,u_1,...$ (i.e. a joint torque sequence), the _solution_ to this discrete ODE is a state __sequence__ $x_0, x_1, ...$ (i.e. joint position and velocity sequences). We will use this discretized model to simulate robot, that is to calculate the sequence of $x_n$'s given an initial state $x_0$ and a finite control sequence $u_0,...,u_N$.   

__Question__ : Say we chose some $\Delta t$ and a number of simulation steps $N$. What does the quantity $N\Delta t$ represent?


__Question__ : Implement a member function `step` in the class `DynamicModel1R` that implements the discrete time ODE shown above. It should take the current state $x_n$, control $u_n$ and discretization step $\Delta t$ as an input and return the next state $x_{n+1}$.

__Hint__ : In your implementation, use the DDM.

In [4]:
class DynamicModel1R:
    
''' !!! Copy-paste here the functions you implemented previously : __init__, DDM, IDM !!! '''

    def step(self, q_n, v_n, tau_n, dt):
        '''
        This function integrates the dynamics of the manipulator for one time step of duration dt 
        using the Euler discretization method
        Input:
            q_n   : joint position at current time step n
            v_n   : joint velocity at current time step n
            tau_n : torque applied at the end of manipulator at current time step n
            dt    : Euler discretization step (the "Delta t")
        Output:
            q_np1 : next position
            v_np1 : next velocity
        '''
        pass

IndentationError: expected an indented block (<ipython-input-4-d98ecb41d796>, line 3)

## Simulation of the 1R robot

Now we have all the ingredients to simulate the 1R robot ! 

__Question__ : Using the `DynamicModel1R.step` function you just implemented, write a python script that 
- instantiates the `DynamicModel1R` class 
- given some initial state $x_0 = [q_0,v_0]^T$ of your choice, it simulates a user-defined control sequence $u_0,u_2...,u_N$ where $N$ is the total number of steps in the trajectory
- stores the trajectory obtained with the simulation $x_0=[q_0,v_0]^T, ..., x_{N+1}=[q_{N+1},v_{N+1}]^T$ (output of the `step` function) 
- plots nicely the control and state trajectories with legend, titles, axis titles, etc.

Try first with a zero torque sequence, i.e. $u_0,...,u_N = 0,...,0$. Set $\Delta t=0.001s$ and $N=10000$ ($10s$ simulation). Then feel free to try out different values of $N$ and $\Delta t$ and to change the control sequence.

__Hint__ : Use a `for` loop on $N$ and call `DynamicModel1R.step` at each cycle.

In [6]:
# Create robot model

# SIMULATION

# PLOTS

__Bonus question__ : Set a larger $N$ (e.g. $10^6$), what do you observe ? What is happening ?

## Simulation using Inverse Dynamics 

Now the torque sent to the 1R robot doesn't seem to achieve anything special. This is because we input "random" joint torques... What if we want to generate, say, a circle trajectory as we did using the kinematic planner ? Which torque to sequence to use in this case ? The IDM can be used for this purpose: this technique is called __inverse dynamics__. It gives us the torque $\tau$ that must be applied to achieve a desired motion $q,\dot{q},\ddot{q}$.

__Question__ : Using the class `DynamicModel1R`, write a script that : 
- 1. Calculates the sequence of torques realizing a circle trajectory with $\ddot q = 0$.

__Hint__: Use the `Model1R` and `KinematicPlanner` classes from the previous Notebook to determine the desired $q,\dot{q}$.

- 2. Simulates the obtained sequence of torque and plot everything 

__Hint__: Re-use most of the simulation and plotting scripts from the previous question

In [6]:
# Using results from the previous notebook
  # Colab
# from google.colab import drive
# drive.mount('/content/drive')
# import sys
# sys.path.append('/content/drive/My Drive')
from models import Model1R, KinematicPlanner

# Simulation parameters (N,dt)

# PLAN TRAJECTORY using KINEMATICS

# FIND TORQUES using INVERSE DYNAMICS

# SIMULATION
    
# PLOTS